In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

tf.random.set_seed(42)
np.random.seed(42)

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU available: {len(tf.config.list_physical_devices('GPU')) > 0}")

In [ ]:
# Load all four datasets (different gate voltages)
# VGS: Tek023=4.0V, Tek024=4.5V, Tek025=5.0V, Tek031=5.5V
datasets = {
    "Tek023 (VGS=4.0V)": pd.read_csv("data/0901_Tek023.csv"),
    "Tek024 (VGS=4.5V)": pd.read_csv("data/0901_Tek024.csv"),
    "Tek025 (VGS=5.0V)": pd.read_csv("data/0901_Tek025.csv"),
    "Tek031 (VGS=5.5V)": pd.read_csv("data/0901_Tek031.csv"),
}

# Also load interpolated versions
datasets_interp = {
    "Tek023 (VGS=4.0V)": pd.read_csv("data/0901_Tek023_interpolated.csv"),
    "Tek024 (VGS=4.5V)": pd.read_csv("data/0901_Tek024_interpolated.csv"),
    "Tek025 (VGS=5.0V)": pd.read_csv("data/0901_Tek025_interpolated.csv"),
    "Tek031 (VGS=5.5V)": pd.read_csv("data/0901_Tek031_interpolated.csv"),
}

# Quick summary
for name, df in datasets.items():
    vth_0 = df["vth_average"].iloc[0]
    vth_end = df["vth_average"].iloc[-1]
    print(f"{name}: {len(df)} points, Vth_0={vth_0:.4f}V, Vth_end={vth_end:.4f}V, "
          f"ΔVth={vth_end - vth_0:.4f}V")

In [ ]:
epsilon = 1e-9

def prepare_data(df_raw, df_interp, train_t_max=1000.0, train_t_min=1.0):
    """Prepare training/test splits from raw and interpolated data."""
    # Use interpolated data for training (dense), raw data for test evaluation
    train_mask = (df_interp["t"] >= train_t_min) & (df_interp["t"] <= train_t_max)
    
    t_train = df_interp["t"][train_mask].values
    log_t_train = df_interp["log_t"][train_mask].values
    vth_train = df_interp["vth_average"][train_mask].values
    
    # Test on raw (original) data points beyond training window
    test_mask_raw = df_raw["t"] > train_t_max
    t_test = df_raw["t"][test_mask_raw].values
    log_t_test = np.log(t_test + epsilon)
    vth_test = df_raw["vth_average"][test_mask_raw].values
    
    # Initial condition from first measured point
    vth_0 = df_raw["vth_average"].iloc[0]
    
    # Observed saturation value (max Vth in full dataset, used as physics prior)
    vth_max_obs = df_raw["vth_average"].max()
    
    return {
        "t_train": t_train, "log_t_train": log_t_train, "vth_train": vth_train,
        "t_test": t_test, "log_t_test": log_t_test, "vth_test": vth_test,
        "vth_0": vth_0, "vth_max_obs": vth_max_obs,
    }

# Prepare primary dataset (Tek023)
data_primary = prepare_data(
    datasets["Tek023 (VGS=4.0V)"],
    datasets_interp["Tek023 (VGS=4.0V)"]
)

print(f"Training: {len(data_primary['t_train'])} points (t in [1, 1000]s)")
print(f"Test: {len(data_primary['t_test'])} points (t > 1000s)")
print(f"Vth_0 = {data_primary['vth_0']:.4f}V")
print(f"Vth_max (observed) = {data_primary['vth_max_obs']:.4f}V")

In [ ]:
# =============================================================================
# SCALING AND TENSOR PREPARATION
# =============================================================================

d = data_primary

# Scale log(t) and Vth to [0, 1] for stable training
scaler_t = MinMaxScaler()
scaler_vth = MinMaxScaler()

log_t_train_scaled = scaler_t.fit_transform(d["log_t_train"].reshape(-1, 1))
log_t_test_scaled = scaler_t.transform(d["log_t_test"].reshape(-1, 1))

vth_train_scaled = scaler_vth.fit_transform(d["vth_train"].reshape(-1, 1))

# Scale the physics constants into the same space
vth_0_scaled = scaler_vth.transform([[d["vth_0"]]])[0, 0]
vth_max_scaled = scaler_vth.transform([[d["vth_max_obs"]]])[0, 0]

# Convert to TF tensors
t_train_tf = tf.constant(log_t_train_scaled, dtype=tf.float32)
t_test_tf = tf.constant(log_t_test_scaled, dtype=tf.float32)
vth_train_tf = tf.constant(vth_train_scaled, dtype=tf.float32)

# Collocation points: log-spaced grid from train_t_min to 10000s for physics enforcement
# This extends BEYOND the training window -- the physics constrains extrapolation
t_colloc_raw = np.geomspace(1.0, 10000.0, 512)
log_t_colloc = np.log(t_colloc_raw + epsilon)
log_t_colloc_scaled = scaler_t.transform(log_t_colloc.reshape(-1, 1))
t_colloc_tf = tf.constant(log_t_colloc_scaled, dtype=tf.float32)

# Also need unscaled time on collocation grid (for physics ODE which uses real time)
t_colloc_real_tf = tf.constant(t_colloc_raw.reshape(-1, 1), dtype=tf.float32)

print(f"Collocation points: {len(t_colloc_raw)} (spanning 1s to 10000s)")
print(f"Vth_0 (scaled): {vth_0_scaled:.4f}")
print(f"Vth_max (scaled): {vth_max_scaled:.4f}")

## Physics-Informed Loss: Charge Storage in p-GaN Gate

### Physical Mechanism
Under switching stress, charges accumulate in the p-GaN gate stack of the HEMT:
- **Gate stress** (VGS > 0): holes spill over the AlGaN barrier; electrons from 2DEG are injected
- **Drain stress** (VDS high): impurity ionization in AlGaN; holes exit p-GaN via Schottky junction
- **Net effect**: Negative charges stored in gate stack → positive $V_{th}$ shift

The fundamental relationship between stored charge and threshold voltage shift is:

$$\Delta V_{th}(t) = \frac{Q_{trapped}(t)}{C_{gate}}$$

where $Q_{trapped}(t)$ evolves according to **charge-trapping kinetics** (first-order trap filling with finite trap density).

### Governing ODE Models

We implement three physics models derived from charge-trapping dynamics:

**Model A — Stretched Exponential (distributed trap energies):**
$$\frac{dV_{th}}{dt} = \frac{\beta}{\tau}\left(\frac{t}{\tau}\right)^{\beta-1} (V_{th,max} - V_{th})$$

This arises when traps have a distribution of energy levels (uniform in energy → distribution of time constants).

**Model B — Power-Law:**
$$t \cdot \frac{dV_{th}}{dt} = n \cdot (V_{th} - V_{th,0})$$

Equivalent to $\Delta V_{th} = A \cdot t^n$. Common for diffusion-reaction limited trap filling ($n \sim 0.15$–$0.3$).

**Model C — Logarithmic (uniform trap energy distribution):**
$$\frac{dV_{th}}{dt} = \frac{A}{t + \tau_0}$$

Arises from Shockley-Read-Hall statistics integrated over a uniform distribution of interface trap energies.

### References
- Wu et al., "Dynamic Threshold Voltage of Low-Voltage Schottky-Type p-GaN Gate HEMT in Soft-Switching Operation" (HKUST)
- Wei et al., "Charge Storage Mechanism of Drain Induced Dynamic Threshold Voltage Shift in p-GaN Gate HEMTs," *IEEE EDL*, 2019
- Kong et al., "Physics-informed Neural Network Approach for Early Degradation Trajectory Prediction," *APEC*, 2025

In [ ]:
# =============================================================================
# LEARNABLE PHYSICS PARAMETERS
# =============================================================================
# These are jointly optimized with the neural network weights, allowing the
# PINN to discover the best-fit physics parameters from data.

# Stretched exponential parameters (Model A)
# beta: stretching exponent (0 < beta < 1); beta=1 is simple exponential
# tau: characteristic trapping time constant (seconds)
phys_beta = tf.Variable(0.3, trainable=True, dtype=tf.float32, name="phys_beta")
phys_log_tau = tf.Variable(np.log(100.0), trainable=True, dtype=tf.float32, name="phys_log_tau")
# Use log(tau) to keep tau positive; tau = exp(phys_log_tau)

# Power-law exponent (Model B)
phys_n = tf.Variable(0.2, trainable=True, dtype=tf.float32, name="phys_n")

# Logarithmic amplitude (Model C)
phys_log_A = tf.Variable(np.log(0.05), trainable=True, dtype=tf.float32, name="phys_log_A")
phys_log_tau0 = tf.Variable(np.log(1.0), trainable=True, dtype=tf.float32, name="phys_log_tau0")

# Saturation Vth (learnable, initialized from observed max)
phys_vth_max = tf.Variable(float(vth_max_scaled) + 0.05, trainable=True, dtype=tf.float32,
                            name="phys_vth_max")

physics_vars = [phys_beta, phys_log_tau, phys_n, phys_log_A, phys_log_tau0, phys_vth_max]

print("Learnable physics parameters initialized:")
print(f"  beta = {phys_beta.numpy():.3f}")
print(f"  tau = {np.exp(phys_log_tau.numpy()):.1f}s")
print(f"  n (power-law) = {phys_n.numpy():.3f}")
print(f"  A (log-law) = {np.exp(phys_log_A.numpy()):.4f}")
print(f"  tau_0 (log-law) = {np.exp(phys_log_tau0.numpy()):.2f}s")
print(f"  Vth_max (scaled) = {phys_vth_max.numpy():.4f}")

In [ ]:
# =============================================================================
# PHYSICS LOSS FUNCTIONS — CHARGE-TRAPPING ODE RESIDUALS
# =============================================================================
# Each function computes the residual of a governing ODE on the collocation grid.
# The PINN is trained to minimize these residuals, enforcing that the NN output
# satisfies the charge-trapping differential equation.
#
# IMPORTANT: We work in SCALED space for Vth but need REAL time for the ODE.
# The chain rule handles the scaling: dVth_scaled/d(log_t_scaled) is what
# GradientTape gives us. We convert appropriately.

def compute_dvth_dt_real(model, t_scaled, t_real):
    """
    Compute dVth/dt in real (unscaled) units using automatic differentiation.
    
    Since the model takes scaled log(t) as input, we use the chain rule:
        dVth/dt = (dVth/d_input) * (d_input/dt)
    where input = scaled(log(t)), so d_input/dt = (1/t) * scale_factor.
    """
    with tf.GradientTape() as tape:
        tape.watch(t_scaled)
        vth_pred = model(t_scaled, training=True)
    
    # dVth_scaled / d(log_t_scaled)
    dvth_d_input = tape.gradient(vth_pred, t_scaled)
    
    # Chain rule: d(log_t_scaled)/dt = (1/t) * (1 / (log_t_max - log_t_min))
    # where the MinMaxScaler range comes from the training data
    log_t_range = scaler_t.data_max_[0] - scaler_t.data_min_[0]
    vth_range = scaler_vth.data_max_[0] - scaler_vth.data_min_[0]
    
    # Convert to real units: dVth_real/dt
    dvth_dt_real = dvth_d_input * (vth_range / log_t_range) * (1.0 / (t_real + epsilon))
    
    return vth_pred, dvth_dt_real


def physics_loss_stretched_exp(model, t_scaled, t_real):
    """
    Model A: Stretched exponential charge-trapping ODE.
    
    dVth/dt = (beta/tau) * (t/tau)^(beta-1) * (Vth_max - Vth)
    
    Physical basis: Distribution of trap energy levels in the p-GaN/AlGaN
    interface leads to a distribution of capture time constants. Integrating
    over this distribution yields stretched-exponential filling kinetics.
    
    Residual: R = dVth/dt - (beta/tau)*(t/tau)^(beta-1)*(Vth_max - Vth) = 0
    """
    vth_pred, dvth_dt = compute_dvth_dt_real(model, t_scaled, t_real)
    
    # Constrain parameters to physical ranges
    beta = tf.clip_by_value(phys_beta, 0.05, 0.95)
    tau = tf.exp(tf.clip_by_value(phys_log_tau, np.log(0.01), np.log(1e6)))
    vth_max = phys_vth_max
    
    # Unscale Vth prediction to real units for the ODE
    vth_real = vth_pred * (scaler_vth.data_max_[0] - scaler_vth.data_min_[0]) + scaler_vth.data_min_[0]
    vth_max_real = vth_max * (scaler_vth.data_max_[0] - scaler_vth.data_min_[0]) + scaler_vth.data_min_[0]
    
    # Expected dVth/dt from stretched exponential
    ratio = t_real / tau
    expected_dvth_dt = (beta / tau) * tf.pow(ratio + epsilon, beta - 1.0) * (vth_max_real - vth_real)
    
    residual = dvth_dt - expected_dvth_dt
    return tf.reduce_mean(tf.square(residual))


def physics_loss_power_law(model, t_scaled, t_real):
    """
    Model B: Power-law ODE.
    
    t * dVth/dt = n * (Vth - Vth_0)
    
    Physical basis: Diffusion-reaction limited trap filling, where the
    power-law exponent n reflects the dominant transport mechanism
    (n ~ 0.25 for 1D diffusion-reaction, n ~ 0.16 for 3D diffusion).
    
    Residual: R = t * dVth/dt - n * (Vth - Vth_0) = 0
    """
    vth_pred, dvth_dt = compute_dvth_dt_real(model, t_scaled, t_real)
    
    n = tf.clip_by_value(phys_n, 0.01, 0.8)
    
    # Unscale
    vth_real = vth_pred * (scaler_vth.data_max_[0] - scaler_vth.data_min_[0]) + scaler_vth.data_min_[0]
    vth_0_real = d["vth_0"]
    
    residual = t_real * dvth_dt - n * (vth_real - vth_0_real)
    return tf.reduce_mean(tf.square(residual))


def physics_loss_logarithmic(model, t_scaled, t_real):
    """
    Model C: Logarithmic ODE.
    
    dVth/dt = A / (t + tau_0)
    
    Physical basis: Uniform distribution of interface trap energies.
    Each trap level has time constant tau_i = tau_0 * exp(E_i / kT).
    Integrating over a uniform energy distribution gives log(t) dependence.
    
    Residual: R = dVth/dt - A / (t + tau_0) = 0
    """
    vth_pred, dvth_dt = compute_dvth_dt_real(model, t_scaled, t_real)
    
    A = tf.exp(tf.clip_by_value(phys_log_A, np.log(1e-6), np.log(10.0)))
    tau_0 = tf.exp(tf.clip_by_value(phys_log_tau0, np.log(1e-3), np.log(1e4)))
    
    expected_dvth_dt = A / (t_real + tau_0)
    
    residual = dvth_dt - expected_dvth_dt
    return tf.reduce_mean(tf.square(residual))


def monotonicity_loss(model, t_scaled):
    """Penalize negative dVth/dt (Vth must increase under stress)."""
    with tf.GradientTape() as tape:
        tape.watch(t_scaled)
        vth_pred = model(t_scaled, training=True)
    dvth = tape.gradient(vth_pred, t_scaled)
    return tf.reduce_mean(tf.square(tf.nn.relu(-dvth)))


def saturation_loss(model, t_scaled):
    """Penalize predictions exceeding physical saturation (finite trap density)."""
    vth_pred = model(t_scaled, training=True)
    violation = tf.nn.relu(vth_pred - phys_vth_max)
    return tf.reduce_mean(tf.square(violation))


def boundary_condition_loss(model, t_initial_scaled, vth_initial_scaled):
    """Enforce Vth(t=0) matches fresh device measurement."""
    vth_pred = model(t_initial_scaled, training=True)
    return tf.reduce_mean(tf.square(vth_pred - vth_initial_scaled))


print("Physics loss functions defined:")
print("  - physics_loss_stretched_exp (Model A: distributed trap energies)")
print("  - physics_loss_power_law (Model B: diffusion-reaction limited)")
print("  - physics_loss_logarithmic (Model C: uniform trap energy dist.)")
print("  - monotonicity_loss (dVth/dt >= 0)")
print("  - saturation_loss (Vth <= Vth_max)")
print("  - boundary_condition_loss (Vth(0) = Vth_0)")

In [ ]:
# =============================================================================
# PINN MODEL DEFINITION
# =============================================================================

def create_pinn_model(hidden_layers=[128, 128, 64, 32]):
    """
    Create a feedforward neural network for PINN.
    
    Uses tanh activation (standard for PINNs — smooth, differentiable,
    and its output range naturally spans negative-to-positive which helps
    with gradient-based physics losses).
    """
    inputs = layers.Input(shape=(1,))
    x = inputs
    
    for units in hidden_layers:
        x = layers.Dense(units, activation='tanh',
                         kernel_initializer='glorot_normal')(x)
    
    outputs = layers.Dense(1)(x)
    
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

pinn_model = create_pinn_model(hidden_layers=[128, 128, 64, 32])
pinn_model.summary()
print(f"\nTotal trainable NN parameters: {pinn_model.count_params()}")
print(f"Total trainable physics parameters: {len(physics_vars)}")

In [ ]:
# =============================================================================
# LOSS WEIGHTS AND PHYSICS MODEL SELECTION
# =============================================================================

# Select which physics ODE to enforce
# Options: "stretched_exp", "power_law", "logarithmic"
PHYSICS_MODEL = "stretched_exp"

physics_loss_fn = {
    "stretched_exp": physics_loss_stretched_exp,
    "power_law": physics_loss_power_law,
    "logarithmic": physics_loss_logarithmic,
}[PHYSICS_MODEL]

# Loss weights — using curriculum learning schedule
# Phase 1 (warm-up): Strong data fitting, gentle physics
# Phase 2 (integration): Balance data and physics  
# Phase 3 (physics emphasis): Strong physics for extrapolation

loss_weights = {
    "lambda_data": 1.0,       # Data fidelity (MSE)
    "lambda_ode": 0.1,        # ODE residual (charge-trapping physics)
    "lambda_bc": 1.0,         # Boundary condition (initial Vth)
    "lambda_mono": 0.5,       # Monotonicity constraint
    "lambda_sat": 0.1,        # Saturation constraint
}

print(f"Physics model: {PHYSICS_MODEL}")
print(f"Loss weights: {loss_weights}")

In [ ]:
# =============================================================================
# CUSTOM TRAINING LOOP WITH CURRICULUM LEARNING
# =============================================================================

optimizer = keras.optimizers.Adam(learning_rate=1e-3)

# All trainable variables: NN weights + physics parameters
all_trainable = pinn_model.trainable_variables + physics_vars

def train_step(t_data, vth_data, t_colloc_scaled, t_colloc_real, lw):
    """
    Single training step for the charge-trapping PINN.
    
    Jointly optimizes NN weights and physics parameters.
    """
    with tf.GradientTape() as tape:
        # 1. Data loss (MSE on training points)
        vth_pred = pinn_model(t_data, training=True)
        L_data = tf.reduce_mean(tf.square(vth_pred - vth_data))
        
        # 2. Physics ODE residual loss (on collocation grid, extending to 10000s)
        L_ode = physics_loss_fn(pinn_model, t_colloc_scaled, t_colloc_real)
        
        # 3. Boundary condition loss
        L_bc = boundary_condition_loss(pinn_model, t_data[:1], vth_data[:1])
        
        # 4. Monotonicity loss (on collocation grid)
        L_mono = monotonicity_loss(pinn_model, t_colloc_scaled)
        
        # 5. Saturation loss (on collocation grid)
        L_sat = saturation_loss(pinn_model, t_colloc_scaled)
        
        # Total weighted loss
        L_total = (lw["lambda_data"] * L_data +
                   lw["lambda_ode"] * L_ode +
                   lw["lambda_bc"] * L_bc +
                   lw["lambda_mono"] * L_mono +
                   lw["lambda_sat"] * L_sat)
    
    gradients = tape.gradient(L_total, all_trainable)
    # Clip gradients to avoid instability
    gradients = [tf.clip_by_norm(g, 1.0) if g is not None else g for g in gradients]
    optimizer.apply_gradients(
        [(g, v) for g, v in zip(gradients, all_trainable) if g is not None]
    )
    
    return {
        "total": L_total, "data": L_data, "ode": L_ode,
        "bc": L_bc, "mono": L_mono, "sat": L_sat,
    }

print("Training step defined. Ready to train.")

In [ ]:
# =============================================================================
# TRAINING WITH CURRICULUM LEARNING
# =============================================================================
# Phase 1: Fit data first (strong data weight, gentle physics)
# Phase 2: Integrate physics (balanced weights)
# Phase 3: Emphasize physics for extrapolation (strong ODE weight)

total_epochs = 8000
print_every = 500

# Curriculum schedule: (epoch_end, weight_overrides)
curriculum = [
    (2000, {"lambda_data": 1.0, "lambda_ode": 0.01, "lambda_bc": 1.0, "lambda_mono": 0.1, "lambda_sat": 0.01}),
    (5000, {"lambda_data": 1.0, "lambda_ode": 0.5,  "lambda_bc": 0.5, "lambda_mono": 0.5, "lambda_sat": 0.1}),
    (8000, {"lambda_data": 1.0, "lambda_ode": 1.0,  "lambda_bc": 0.1, "lambda_mono": 1.0, "lambda_sat": 0.5}),
]

history = {"total": [], "data": [], "ode": [], "bc": [], "mono": [], "sat": []}

# Learning rate schedule
lr_schedule = {0: 1e-3, 2000: 5e-4, 5000: 1e-4}

print(f"Starting PINN training: {PHYSICS_MODEL} model")
print(f"Total epochs: {total_epochs}")
print("=" * 90)

current_phase = 0
for epoch in range(1, total_epochs + 1):
    # Determine current curriculum phase weights
    for phase_idx, (phase_end, phase_weights) in enumerate(curriculum):
        if epoch <= phase_end:
            lw = phase_weights
            if phase_idx != current_phase:
                current_phase = phase_idx
                print(f"\n--- Phase {current_phase + 1} (epoch {epoch}): {phase_weights} ---")
            break
    
    # Learning rate schedule
    if epoch in lr_schedule:
        optimizer.learning_rate.assign(lr_schedule[epoch])
        print(f"  LR -> {lr_schedule[epoch]}")
    
    losses = train_step(t_train_tf, vth_train_tf, t_colloc_tf, t_colloc_real_tf, lw)
    
    for key in history:
        history[key].append(float(losses[key].numpy()))
    
    if epoch % print_every == 0:
        print(f"Epoch {epoch}/{total_epochs}: "
              f"total={history['total'][-1]:.4g} "
              f"data={history['data'][-1]:.4g} "
              f"ode={history['ode'][-1]:.4g} "
              f"mono={history['mono'][-1]:.4g} "
              f"sat={history['sat'][-1]:.4g}")

print("=" * 90)
print("Training complete!")
print(f"\nLearned physics parameters:")
print(f"  beta = {tf.clip_by_value(phys_beta, 0.05, 0.95).numpy():.4f}")
print(f"  tau = {np.exp(phys_log_tau.numpy()):.2f}s")
print(f"  n (power-law) = {tf.clip_by_value(phys_n, 0.01, 0.8).numpy():.4f}")
print(f"  A (log-law) = {np.exp(phys_log_A.numpy()):.6f}")
print(f"  tau_0 (log-law) = {np.exp(phys_log_tau0.numpy()):.4f}s")
print(f"  Vth_max (real) = {phys_vth_max.numpy() * (scaler_vth.data_max_[0] - scaler_vth.data_min_[0]) + scaler_vth.data_min_[0]:.4f}V")

In [ ]:
# =============================================================================
# TRAINING HISTORY VISUALIZATION
# =============================================================================

fig, axes = plt.subplots(2, 3, figsize=(18, 10))

loss_names = ["total", "data", "ode", "bc", "mono", "sat"]
titles = ["Total Loss", "Data Loss (MSE)", "ODE Residual (Physics)",
          "Boundary Condition", "Monotonicity", "Saturation"]

for ax, key, title in zip(axes.flatten(), loss_names, titles):
    vals = history[key]
    ax.semilogy(vals, alpha=0.6)
    # Add smoothed line
    window = min(100, len(vals) // 10)
    if window > 1:
        smoothed = np.convolve(vals, np.ones(window)/window, mode='valid')
        ax.semilogy(range(window-1, len(vals)), smoothed, 'r-', linewidth=2)
    ax.set_title(title)
    ax.set_xlabel("Epoch")
    ax.set_ylabel("Loss")
    # Mark phase transitions
    ax.axvline(x=2000, color='gray', linestyle=':', alpha=0.5, label='Phase 2')
    ax.axvline(x=5000, color='gray', linestyle='--', alpha=0.5, label='Phase 3')

axes[0, 0].legend(fontsize=8)
plt.suptitle(f"PINN Training History — {PHYSICS_MODEL} model", fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# =============================================================================
# EVALUATION
# =============================================================================

def evaluate_pinn(model, data_dict, scaler_t, scaler_vth, label=""):
    """Evaluate PINN predictions on train and test sets."""
    d = data_dict
    
    # Predict on training data
    log_t_train_sc = scaler_t.transform(d["log_t_train"].reshape(-1, 1))
    vth_pred_train_sc = model.predict(log_t_train_sc, verbose=0)
    vth_pred_train = scaler_vth.inverse_transform(vth_pred_train_sc).flatten()
    
    # Predict on test data
    log_t_test_sc = scaler_t.transform(d["log_t_test"].reshape(-1, 1))
    vth_pred_test_sc = model.predict(log_t_test_sc, verbose=0)
    vth_pred_test = scaler_vth.inverse_transform(vth_pred_test_sc).flatten()
    
    # Metrics
    def metrics(y_true, y_pred):
        return {
            "MSE": mean_squared_error(y_true, y_pred),
            "RMSE": np.sqrt(mean_squared_error(y_true, y_pred)),
            "MAE": mean_absolute_error(y_true, y_pred),
            "R2": r2_score(y_true, y_pred) if len(y_true) >= 2 else np.nan,
        }
    
    train_m = metrics(d["vth_train"], vth_pred_train)
    test_m = metrics(d["vth_test"], vth_pred_test)
    
    # Endpoint prediction at ~10000s
    idx_10000 = np.argmin(np.abs(d["t_test"] - 10000.0))
    vth_actual_10000 = d["vth_test"][idx_10000]
    vth_pred_10000 = vth_pred_test[idx_10000]
    
    print(f"\n{'='*60}")
    print(f"PINN Evaluation {label}")
    print(f"{'='*60}")
    print(f"Training (t <= 1000s): MSE={train_m['MSE']:.6f}, RMSE={train_m['RMSE']:.6f}")
    print(f"Test (t > 1000s):      MSE={test_m['MSE']:.6f}, RMSE={test_m['RMSE']:.6f}, "
          f"MAE={test_m['MAE']:.6f}")
    print(f"\nPrediction at t=10000s:")
    print(f"  Actual:    {vth_actual_10000:.6f}V")
    print(f"  Predicted: {vth_pred_10000:.6f}V")
    print(f"  Error:     {abs(vth_actual_10000 - vth_pred_10000):.6f}V")
    
    return {
        "vth_pred_train": vth_pred_train, "vth_pred_test": vth_pred_test,
        "train_metrics": train_m, "test_metrics": test_m,
        "vth_10000_actual": vth_actual_10000, "vth_10000_pred": vth_pred_10000,
    }

results = evaluate_pinn(pinn_model, data_primary, scaler_t, scaler_vth,
                         label=f"({PHYSICS_MODEL})")

In [ ]:
# =============================================================================
# PREDICTION PLOTS: TRAIN vs TEST
# =============================================================================
d = data_primary
r = results

fig, axes = plt.subplots(1, 2, figsize=(16, 5))

# Training range
axes[0].plot(d["log_t_train"], d["vth_train"], 'k-', linewidth=2, label="Actual")
axes[0].plot(d["log_t_train"], r["vth_pred_train"], 'r--', alpha=0.8, linewidth=2, label="PINN")
axes[0].set_xlabel("log(Time)")
axes[0].set_ylabel("Vth (V)")
axes[0].set_title("Training Range (t <= 1000s)")
axes[0].legend()

# Test range (extrapolation)
axes[1].plot(d["log_t_test"], d["vth_test"], 'k-', linewidth=2, label="Actual")
axes[1].plot(d["log_t_test"], r["vth_pred_test"], 'r--', alpha=0.8, linewidth=2, label="PINN")
axes[1].set_xlabel("log(Time)")
axes[1].set_ylabel("Vth (V)")
axes[1].set_title("Extrapolation (t > 1000s)")
axes[1].legend()

plt.suptitle(f"Charge-Trapping PINN ({PHYSICS_MODEL})", fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# =============================================================================
# FULL RANGE PREDICTION — with physics-implied trajectory
# =============================================================================

# Generate a dense prediction curve from t=1s to t=20000s (extrapolation beyond data)
t_dense = np.geomspace(1.0, 20000.0, 1000)
log_t_dense = np.log(t_dense + epsilon)
log_t_dense_scaled = scaler_t.transform(log_t_dense.reshape(-1, 1))
vth_dense_scaled = pinn_model.predict(log_t_dense_scaled, verbose=0)
vth_dense = scaler_vth.inverse_transform(vth_dense_scaled).flatten()

# Also compute the analytical stretched-exp curve with learned parameters for comparison
beta_val = tf.clip_by_value(phys_beta, 0.05, 0.95).numpy()
tau_val = np.exp(phys_log_tau.numpy())
vth_max_real = phys_vth_max.numpy() * (scaler_vth.data_max_[0] - scaler_vth.data_min_[0]) + scaler_vth.data_min_[0]
vth_0_real = d["vth_0"]
vth_analytical = vth_0_real + (vth_max_real - vth_0_real) * (1 - np.exp(-(t_dense / tau_val)**beta_val))

plt.figure(figsize=(14, 6))

# Actual data
df_raw = datasets["Tek023 (VGS=4.0V)"]
plt.scatter(np.log(df_raw["t"] + epsilon), df_raw["vth_average"],
            color='black', s=30, zorder=5, label="Measured Data")

# PINN prediction
plt.plot(log_t_dense, vth_dense, 'r-', linewidth=2, label=f"PINN ({PHYSICS_MODEL})")

# Analytical curve from learned parameters
plt.plot(log_t_dense, vth_analytical, 'b--', linewidth=1.5, alpha=0.7,
         label=f"Analytical (beta={beta_val:.3f}, tau={tau_val:.1f}s)")

# Markers
plt.axvline(x=np.log(1000), color='gray', linestyle=':', alpha=0.7, label="Train/Test split (1000s)")
plt.axvline(x=np.log(10000), color='gray', linestyle='--', alpha=0.5, label="Target (10000s)")

# Saturation line
plt.axhline(y=vth_max_real, color='green', linestyle=':', alpha=0.5,
            label=f"Learned Vth_max = {vth_max_real:.4f}V")

plt.xlabel("log(Time)")
plt.ylabel("Vth (V)")
plt.title(f"Charge-Trapping PINN: Full Trajectory Prediction (Tek023, VGS=4.0V)")
plt.legend(loc="lower right", fontsize=9)
plt.tight_layout()
plt.show()

print(f"\nPhysics interpretation:")
print(f"  The network learned a trap-filling process with:")
print(f"  - Stretching exponent beta = {beta_val:.4f} (0=very distributed, 1=single trap)")
print(f"  - Characteristic time tau = {tau_val:.2f}s")
print(f"  - Saturation Vth = {vth_max_real:.4f}V")

In [ ]:
# =============================================================================
# COMPARISON: PINN vs KONG-STYLE (LSTM+PHYSICS) vs BASELINES
# =============================================================================
# Compare this charge-trapping PINN against the models from standardmodel.ipynb

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor

df_raw = datasets["Tek023 (VGS=4.0V)"]
t_all = df_raw["t"].astype(float).values
y_all = df_raw["vth_average"].astype(float).values

train_mask = (t_all >= 1.0) & (t_all <= 1000.0)
test_mask = t_all > 1000.0

X_train_bl = np.log(t_all[train_mask] + epsilon).reshape(-1, 1)
y_train_bl = y_all[train_mask]
X_test_bl = np.log(t_all[test_mask] + epsilon).reshape(-1, 1)
y_test_bl = y_all[test_mask]

# Baseline models
lin = LinearRegression().fit(X_train_bl, y_train_bl)
rf = RandomForestRegressor(n_estimators=300, random_state=42).fit(X_train_bl, y_train_bl)
bl_scaler = StandardScaler()
X_train_bl_s = bl_scaler.fit_transform(X_train_bl)
X_test_bl_s = bl_scaler.transform(X_test_bl)
mlp = MLPRegressor(hidden_layer_sizes=(64, 32), max_iter=5000, random_state=42)
mlp.fit(X_train_bl_s, y_train_bl)

# Predictions at 10000s
idx_10000 = np.argmin(np.abs(t_all[test_mask] - 10000.0))
y_actual_10000 = y_test_bl[idx_10000]

comparison = []
for name, pred_fn in [
    ("Linear Regression", lambda X: lin.predict(X)),
    ("Random Forest", lambda X: rf.predict(X)),
    ("MLP", lambda X: mlp.predict(bl_scaler.transform(X))),
]:
    y_pred = pred_fn(X_test_bl)
    y_pred_10000 = y_pred[idx_10000]
    comparison.append({
        "Model": name,
        "AbsErr@10000s": abs(y_actual_10000 - y_pred_10000),
        "Test MAE": mean_absolute_error(y_test_bl, y_pred),
        "Test RMSE": np.sqrt(mean_squared_error(y_test_bl, y_pred)),
    })

# PINN result
comparison.append({
    "Model": f"PINN ({PHYSICS_MODEL})",
    "AbsErr@10000s": abs(results["vth_10000_actual"] - results["vth_10000_pred"]),
    "Test MAE": results["test_metrics"]["MAE"],
    "Test RMSE": results["test_metrics"]["RMSE"],
})

# LSTM+Physics from standardmodel.ipynb (manually entered from previous runs)
comparison.append({
    "Model": "LSTM+Physics (Kong-style)",
    "AbsErr@10000s": 0.000556,  # From standardmodel.ipynb results
    "Test MAE": np.nan,  # Would need to re-run
    "Test RMSE": np.sqrt(0.001473),
})

comp_df = pd.DataFrame(comparison)
print("=" * 70)
print("MODEL COMPARISON: Error at t=10000s (train on t<=1000s)")
print("=" * 70)
print(comp_df.sort_values("AbsErr@10000s").to_string(index=False))

# Bar chart
fig, ax = plt.subplots(figsize=(10, 5))
comp_sorted = comp_df.sort_values("AbsErr@10000s")
colors = ['tab:red' if 'PINN' in m else 'tab:purple' if 'LSTM' in m else 'steelblue'
          for m in comp_sorted["Model"]]
ax.barh(comp_sorted["Model"], comp_sorted["AbsErr@10000s"], color=colors)
ax.set_xlabel("|Vth Error| at t=10000s (V)")
ax.set_title("Extrapolation Accuracy Comparison (train on t<=1000s)")
plt.tight_layout()
plt.show()

## Cross-Dataset Validation

Run the charge-trapping PINN on all four datasets (different VGS conditions) to verify
that the physics model generalizes. The learned parameters (beta, tau, Vth_max) should
vary systematically with gate voltage — higher VGS should produce more trapped charge
and potentially different time constants.

In [ ]:
# =============================================================================
# CROSS-DATASET VALIDATION
# =============================================================================
# Train separate PINNs on each dataset and compare learned physics parameters

def train_pinn_on_dataset(df_raw, df_interp, dataset_name, physics_model="stretched_exp",
                           total_epochs=6000, verbose=True):
    """Train a fresh PINN on a single dataset and return results."""
    tf.random.set_seed(42)
    np.random.seed(42)
    
    # Prepare data
    dd = prepare_data(df_raw, df_interp)
    
    # Scalers
    sc_t = MinMaxScaler()
    sc_vth = MinMaxScaler()
    
    lt_train_sc = sc_t.fit_transform(dd["log_t_train"].reshape(-1, 1))
    lt_test_sc = sc_t.transform(dd["log_t_test"].reshape(-1, 1))
    vth_train_sc = sc_vth.fit_transform(dd["vth_train"].reshape(-1, 1))
    
    t_train_t = tf.constant(lt_train_sc, dtype=tf.float32)
    vth_train_t = tf.constant(vth_train_sc, dtype=tf.float32)
    
    # Collocation
    tc_raw = np.geomspace(1.0, 10000.0, 512)
    lt_colloc_sc = sc_t.transform(np.log(tc_raw + epsilon).reshape(-1, 1))
    tc_sc_t = tf.constant(lt_colloc_sc, dtype=tf.float32)
    tc_real_t = tf.constant(tc_raw.reshape(-1, 1), dtype=tf.float32)
    
    # Fresh model + physics params
    model = create_pinn_model(hidden_layers=[128, 128, 64, 32])
    p_beta = tf.Variable(0.3, trainable=True, dtype=tf.float32)
    p_log_tau = tf.Variable(np.log(100.0), trainable=True, dtype=tf.float32)
    p_vth_max = tf.Variable(
        float(sc_vth.transform([[dd["vth_max_obs"]]])[0, 0]) + 0.05,
        trainable=True, dtype=tf.float32
    )
    
    opt = keras.optimizers.Adam(learning_rate=1e-3)
    all_vars = model.trainable_variables + [p_beta, p_log_tau, p_vth_max]
    
    def step(lw):
        with tf.GradientTape() as tape:
            vth_p = model(t_train_t, training=True)
            L_data = tf.reduce_mean(tf.square(vth_p - vth_train_t))
            
            # Stretched exp ODE on collocation
            with tf.GradientTape() as tape2:
                tape2.watch(tc_sc_t)
                vth_c = model(tc_sc_t, training=True)
            dvth_dinput = tape2.gradient(vth_c, tc_sc_t)
            
            lr = sc_t.data_max_[0] - sc_t.data_min_[0]
            vr = sc_vth.data_max_[0] - sc_vth.data_min_[0]
            dvth_dt = dvth_dinput * (vr / lr) * (1.0 / (tc_real_t + epsilon))
            
            beta = tf.clip_by_value(p_beta, 0.05, 0.95)
            tau = tf.exp(tf.clip_by_value(p_log_tau, np.log(0.01), np.log(1e6)))
            vth_real = vth_c * vr + sc_vth.data_min_[0]
            vm_real = p_vth_max * vr + sc_vth.data_min_[0]
            
            ratio = tc_real_t / tau
            expected = (beta / tau) * tf.pow(ratio + epsilon, beta - 1.0) * (vm_real - vth_real)
            L_ode = tf.reduce_mean(tf.square(dvth_dt - expected))
            
            # Monotonicity
            with tf.GradientTape() as tape3:
                tape3.watch(tc_sc_t)
                vth_m = model(tc_sc_t, training=True)
            dvm = tape3.gradient(vth_m, tc_sc_t)
            L_mono = tf.reduce_mean(tf.square(tf.nn.relu(-dvm)))
            
            # BC
            vth_bc = model(t_train_t[:1], training=True)
            L_bc = tf.reduce_mean(tf.square(vth_bc - vth_train_t[:1]))
            
            L_total = (lw["d"] * L_data + lw["o"] * L_ode +
                       lw["m"] * L_mono + lw["b"] * L_bc)
        
        grads = tape.gradient(L_total, all_vars)
        grads = [tf.clip_by_norm(g, 1.0) if g is not None else g for g in grads]
        opt.apply_gradients([(g, v) for g, v in zip(grads, all_vars) if g is not None])
        return float(L_total.numpy()), float(L_data.numpy())
    
    # Training with curriculum
    phases = [
        (total_epochs // 3, {"d": 1.0, "o": 0.01, "m": 0.1, "b": 1.0}),
        (2 * total_epochs // 3, {"d": 1.0, "o": 0.5, "m": 0.5, "b": 0.5}),
        (total_epochs, {"d": 1.0, "o": 1.0, "m": 1.0, "b": 0.1}),
    ]
    
    for phase_end, lw in phases:
        for ep in range(phase_end - (phases[0][0] if phase_end > phases[0][0] else 0), phase_end):
            lt, ld = step(lw)
        if verbose:
            print(f"  [{dataset_name}] epoch {phase_end}: total={lt:.4g}, data={ld:.4g}")
    
    if total_epochs > phases[0][0]:
        opt.learning_rate.assign(5e-4)
    
    # Evaluate
    vth_pred_test_sc = model.predict(lt_test_sc, verbose=0)
    vth_pred_test = sc_vth.inverse_transform(vth_pred_test_sc).flatten()
    
    idx_10000 = np.argmin(np.abs(dd["t_test"] - 10000.0))
    err_10000 = abs(dd["vth_test"][idx_10000] - vth_pred_test[idx_10000])
    test_mae = mean_absolute_error(dd["vth_test"], vth_pred_test)
    
    beta_val = tf.clip_by_value(p_beta, 0.05, 0.95).numpy()
    tau_val = np.exp(p_log_tau.numpy())
    vm_val = p_vth_max.numpy() * (sc_vth.data_max_[0] - sc_vth.data_min_[0]) + sc_vth.data_min_[0]
    
    return {
        "dataset": dataset_name,
        "beta": beta_val, "tau": tau_val, "Vth_max": vm_val,
        "AbsErr@10000s": err_10000, "Test_MAE": test_mae,
        "vth_pred_test": vth_pred_test, "dd": dd, "model": model,
    }

# Train on all four datasets
cross_results = []
for name in datasets.keys():
    print(f"\nTraining on {name}...")
    res = train_pinn_on_dataset(datasets[name], datasets_interp[name], name)
    cross_results.append(res)

# Summary table
cross_df = pd.DataFrame([{
    "Dataset": r["dataset"],
    "beta": f"{r['beta']:.4f}",
    "tau (s)": f"{r['tau']:.2f}",
    "Vth_max (V)": f"{r['Vth_max']:.4f}",
    "|Err|@10000s": f"{r['AbsErr@10000s']:.6f}",
    "Test MAE": f"{r['Test_MAE']:.6f}",
} for r in cross_results])

print("\n" + "=" * 80)
print("CROSS-DATASET RESULTS: Charge-Trapping PINN (stretched exponential)")
print("=" * 80)
print(cross_df.to_string(index=False))

In [ ]:
# =============================================================================
# CROSS-DATASET VISUALIZATION
# =============================================================================

fig, axes = plt.subplots(2, 2, figsize=(16, 12))

for ax, res in zip(axes.flatten(), cross_results):
    dd = res["dd"]
    
    # Plot actual (raw) data
    df_raw = [v for k, v in datasets.items() if k == res["dataset"]][0]
    ax.scatter(np.log(df_raw["t"] + epsilon), df_raw["vth_average"],
               color='black', s=25, zorder=5, label="Measured")
    
    # Plot PINN prediction on test
    ax.plot(dd["log_t_test"], res["vth_pred_test"], 'r-', linewidth=2,
            label=f"PINN (err@10k={res['AbsErr@10000s']:.4f}V)")
    
    ax.axvline(x=np.log(1000), color='gray', linestyle=':', alpha=0.7)
    ax.set_xlabel("log(Time)")
    ax.set_ylabel("Vth (V)")
    ax.set_title(f"{res['dataset']}\nbeta={res['beta']:.3f}, tau={res['tau']:.1f}s, "
                 f"Vth_max={res['Vth_max']:.3f}V")
    ax.legend(fontsize=9)

plt.suptitle("Charge-Trapping PINN: Cross-Dataset Validation", fontsize=14)
plt.tight_layout()
plt.show()

# Physics parameter trends
fig, axes = plt.subplots(1, 3, figsize=(15, 4))
vgs_labels = ["4.0V", "4.5V", "5.0V", "5.5V"]

axes[0].bar(vgs_labels, [r["beta"] for r in cross_results], color="steelblue")
axes[0].set_ylabel("beta")
axes[0].set_xlabel("VGS")
axes[0].set_title("Stretching Exponent vs Gate Voltage")

axes[1].bar(vgs_labels, [r["tau"] for r in cross_results], color="coral")
axes[1].set_ylabel("tau (s)")
axes[1].set_xlabel("VGS")
axes[1].set_title("Trapping Time Constant vs Gate Voltage")

axes[2].bar(vgs_labels, [r["Vth_max"] for r in cross_results], color="green")
axes[2].set_ylabel("Vth_max (V)")
axes[2].set_xlabel("VGS")
axes[2].set_title("Saturation Vth vs Gate Voltage")

plt.suptitle("Learned Physics Parameters Across Gate Voltages", fontsize=14)
plt.tight_layout()
plt.show()